In [1]:
import numpy as np
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton
from datetime import datetime

## 추천받은 포트폴리오 -> 잔고(detail 테이블)

In [2]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')
price_db = pd.read_pickle('../../data/external/price_db_d.pkl')
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')    

In [98]:
bal = pd.read_pickle('../../data/processed/balance_s.pkl')

In [104]:
bal.iloc[pd.to_datetime(bal.date).values.argsort()[::-1]]

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
6,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A114260,KODEX 국고채3년,2,58120,58120,61026.00,122052.00,0.171652,20210322 16:00채권,N
5,3/22/2021 4:00:00 PM,A001,투자자1,Alternative,A329750,TIGER 미국달러단기채권액티브,16,9645,38580,9645.00,154320.00,0.217033,20210322 16:00대체,N
4,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A302190,TIGER 중장기국채,3,52365,52365,54459.60,163378.80,0.229773,20210322 16:00채권,N
3,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A196230,KBSTAR 단기통안채,1,104325,104325,101195.25,101195.25,0.142319,20210322 16:00채권,N
2,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A360200,KINDEX 미국S&P500,4,10865,43460,10321.75,41287.00,0.058065,20210322 16:00주식,N
1,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A069500,KODEX 200,3,42095,42095,42936.90,128810.70,0.181157,20210322 16:00주식,N
0,3/21/2021 4:00:00 PM,A001,투자자1,Cash,C000001,현금,700000,1,700000,1.00,700000.00,1.000000,20210321 16:00현금성,Y


In [101]:
bal.sort_values('date', ascending=False)

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
1,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A069500,KODEX 200,3,42095,42095,42936.90,128810.70,0.181157,20210322 16:00주식,N
2,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A360200,KINDEX 미국S&P500,4,10865,43460,10321.75,41287.00,0.058065,20210322 16:00주식,N
3,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A196230,KBSTAR 단기통안채,1,104325,104325,101195.25,101195.25,0.142319,20210322 16:00채권,N
4,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A302190,TIGER 중장기국채,3,52365,52365,54459.60,163378.80,0.229773,20210322 16:00채권,N
5,3/22/2021 4:00:00 PM,A001,투자자1,Alternative,A329750,TIGER 미국달러단기채권액티브,16,9645,38580,9645.00,154320.00,0.217033,20210322 16:00대체,N
6,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A114260,KODEX 국고채3년,2,58120,58120,61026.00,122052.00,0.171652,20210322 16:00채권,N
0,3/21/2021 4:00:00 PM,A001,투자자1,Cash,C000001,현금,700000,1,700000,1.00,700000.00,1.000000,20210321 16:00현금성,Y


In [80]:
max_dt = advised_pf.loc[advised_pf.risk_profile==2, ['date']].max().date
latest_pf = advised_pf.loc[(advised_pf.risk_profile==2) & (advised_pf.date==max_dt), :]

In [90]:
from pandas import to_numeric

In [94]:
to_numeric(latest_pf.price.astype(str)).astype(int).apply(lambda x : "{:,}".format(x))

18449     57,900
18450    101,185
18451    100,975
18452    108,565
18453    104,340
18454     41,855
18455     17,750
18456     11,750
Name: price, dtype: object

In [75]:
to = 3.332

In [77]:
'{:,}'.format(333322)

'333,322'

In [78]:
'{:,}'.format(int(to))

'3'

In [79]:
latest_pf.loc[:, ['price', 'volume']].astype(int).applymap(lambda x : "{:,}".format(x))

,price,volume
18449,"57,900","1,893"
18450,"101,185","137,782"
18451,"100,975","490,574"
18452,"108,565","3,011"
18453,"104,340","180,070"
18454,"41,855","9,277,711"
18455,"17,750","23,246"
18456,"11,750","9,292"


In [47]:
latest_pf

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
18449,2021-03-19,2,A114260,0.240,KTB_D3,KODEX 국고채3년,"57,900",1893.0,825.848869,Fixed Income
18450,2021-03-19,2,A122260,0.240,BOK_D1,KOSEF 통안채1년,"101,185",137782.0,47097.865417,Fixed Income
18451,2021-03-19,2,A130730,0.240,MM,KOSEF 단기자금,"100,975",490574.0,64342.722020,Fixed Income
18452,2021-03-19,2,A273130,0.095,ACT_BOND_D4,KODEX 종합채권(AA-이상)액티브,"108,565",3011.0,2012.456380,Fixed Income
18453,2021-03-19,2,A196230,0.065,BOK_D.4,KBSTAR 단기통안채,"104,340",180070.0,56905.971755,Fixed Income
18454,2021-03-19,2,A069500,0.061,K200,KODEX 200,"41,855",9277711.0,421471.700934,Equity
18455,2021-03-19,2,A251350,0.039,MSCI_WO,KODEX 선진국MSCI World,"17,750",23246.0,3022.359487,Equity
18456,2021-03-19,2,A237370,0.020,BAL_KO3KTB7,KODEX 배당성장채권혼합,"11,750",9292.0,105.542692,Alternative


In [46]:
latest_pf.price = latest_pf.price.astype(int).apply(lambda x : "{:,}".format(x))
latest_pf.weights = latest_pf.weights.apply(lambda x : "{:.3f}".format(x))

/Users/msyeom/anaconda3/envs/ra/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
latest_pf.loc[latest_pf.itemcode=='A122260', 'itemcode'].iloc[0]

'A122260'

In [11]:
next_detail = rebalance(rebal_date=dt, price_d=price_d, detail=copy.deepcopy(next_detail), new_port=new_port)

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
18449,2021-03-19,2,A114260,0.240000,KTB_D3,KODEX 국고채3년,57900.0,1893.0,825.848869,Fixed Income
18450,2021-03-19,2,A122260,0.240000,BOK_D1,KOSEF 통안채1년,101185.0,137782.0,47097.865417,Fixed Income
18451,2021-03-19,2,A130730,0.240000,MM,KOSEF 단기자금,100975.0,490574.0,64342.722020,Fixed Income
18452,2021-03-19,2,A273130,0.094563,ACT_BOND_D4,KODEX 종합채권(AA-이상)액티브,108565.0,3011.0,2012.456380,Fixed Income
18453,2021-03-19,2,A196230,0.065437,BOK_D.4,KBSTAR 단기통안채,104340.0,180070.0,56905.971755,Fixed Income
18454,2021-03-19,2,A069500,0.061392,K200,KODEX 200,41855.0,9277711.0,421471.700934,Equity
18455,2021-03-19,2,A251350,0.038608,MSCI_WO,KODEX 선진국MSCI World,17750.0,23246.0,3022.359487,Equity
18456,2021-03-19,2,A237370,0.020000,BAL_KO3KTB7,KODEX 배당성장채권혼합,11750.0,9292.0,105.542692,Alternative


In [ ]:
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]

In [3]:
userid = 'A00'
username = '투자자0'
current_date = '2020-02-03'
risk_profile = 2

# c_date: 추천 포트폴리오DB에서 사용자가 입력한 날짜와 가장 가까운 날짜.
c_date = advised_pf.loc[advised_pf.date <= current_date, ['date']].max().date
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]
remaining_cash = 135786

In [4]:
port = {}
curr_port = {}
inv_amt = 1e6


start_dt = current_date
end_dt = '2021-2-26'
dates = pd.date_range(start_dt, end_dt)
dates = dates[dates.isin(price_db.date)]  # 거래일만

# 초기 포트폴리오 구성
dt = dates[0].strftime('%Y-%m-%d')
for profile in [2, 3, 4]:
#     curr_port[profile] = get_advised_port(dt, profile)
    price_d = price_db.loc[price_db.date==dt, ['date', 'price', 'itemcode']].reset_index(drop=True)
    port[profile] = advised_pf.loc[
        (advised_pf.date == dt) &
        (advised_pf.risk_profile == profile),
        ['date', 'itemcode', 'weights', 'itemname', 'price', 'asset_class']]
    port[profile]['quantity'] = np.floor(inv_amt*port[profile].weights/port[profile].price)
    port[profile]['value'] = port[profile]['price']*port[profile]['quantity']
    cash = {
        'date':dt,
        'itemcode':'C000001',
        'itemname':'Cash',
        'price':1,
        'asset_class':'Cash',
        'quantity':inv_amt-port[profile]['value'].sum()
    }
    port[profile] = pd.concat((port[profile], pd.DataFrame().from_dict(cash, orient='index').T))
    
    # 현금 포함해서 종목별 평가액과 비중 재계산
    port[profile]['value'] = port[profile]['price']*port[profile]['quantity']
    port[profile]['weights'] = port[profile]['value']/port[profile]['value'].sum()
    port[profile].reset_index(drop=True, inplace=True)
    #port[profile]['date'] = dt
    port[profile]['date'] = datetime.strptime(dt, '%Y-%m-%d')
    port[profile] = pd.merge(port[profile],
             price_d.loc[:, ['date', 'itemcode']],
             left_on=['date', 'itemcode'],
             right_on=['date', 'itemcode'], how='left')
    port[profile]['original'] = 'Y'
    curr_port[profile] = port[profile].loc[port[profile].date==port[profile].date.max(), :]


In [5]:
detail = curr_port[4]

## 리밸런싱

In [30]:
from pypfopt.discrete_allocation import DiscreteAllocation
import copy
from datetime import datetime

In [31]:
# end_date = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].max()
dates = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].unique()

In [32]:
# dt = datetime.strptime('2021-02-01', '%Y-%m-%d')
# dt = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 4:00:00 PM'

In [33]:
dates = advised_pf.loc[(advised_pf.risk_profile == risk_profile) & (
    advised_pf.date >= current_date), 'date'].unique()
rebal_dates = dates[::20]
# dt = rebal_dates[1]

In [34]:
advised_pf.tail()

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
18465,2021-03-19,4,A237370,0.240,BAL_KO3KTB7,KODEX 배당성장채권혼합,11750.0,9292.0,105.542692,Alternative
18466,2021-03-19,4,A157490,0.240,SW,TIGER 소프트웨어,17000.0,188055.0,2484.755538,Equity
18467,2021-03-19,4,A266370,0.136,IT,KODEX IT,21325.0,14939.0,1150.537477,Equity
18468,2021-03-19,4,A069500,0.084,K200,KODEX 200,41855.0,9277711.0,421471.700934,Equity
18469,2021-03-19,4,A114260,0.060,KTB_D3,KODEX 국고채3년,57900.0,1893.0,825.848869,Fixed Income


In [35]:
def rebalance(rebal_date, price_d, detail, new_port):
    '''
    Rebalance a portfolio.
    
    Parameters:
    rebal_date: str
        rebalancing date
      
    detail: DataFrame
      current balance
      
    price_d: DataFrame
      price data on rebal_date
      
    new_port: DataFrame
      A new portfolio. Your current portfolio in `detail` will be rebalanced toward `new_port`.
    '''
    trading_amt = detail.value.sum()
    
    wt = new_port[['itemcode', 'weights']].set_index('itemcode').to_dict()['weights']
    pr = new_port[['itemcode', 'price']].set_index('itemcode').squeeze()
    da = DiscreteAllocation(weights=wt, latest_prices=pr, total_portfolio_value=trading_amt)
    
    allocation, remaining_cash = da.greedy_portfolio()
    print("Discrete allocation on {}:{}".format(rebal_date,allocation))
    print("Funds remaining: {:.2f} KRW".format(remaining_cash))
    
    # 매매한 뒤의 레코드 생성
    df_qty = pd.DataFrame.from_dict(allocation, orient='index', columns=['quantity'])
    next_detail = new_port.merge(df_qty, left_on='itemcode', right_index=True, how='inner')
    next_detail['cost_price'] = next_detail.price.copy()   
    next_detail['cost_value'] = next_detail.cost_price*next_detail.quantity
    next_detail['value'] = next_detail.cost_value.copy()
    
    # 매매하고 남은 돈은 현금으로
    df_cash = {
        'itemcode': 'C000001',
        'quantity': remaining_cash,
        'cost_price': 1,
        'price':1,
        'cost_value': remaining_cash,
        'value': remaining_cash,
        'itemname': '현금',
        'asset_class': 'Cash'
    }
    df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T
    
    next_detail = pd.concat((next_detail[['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
       'itemname', 'asset_class']], df_cash), axis=0)
    
    next_detail['wt'] = next_detail.value/next_detail.value.sum()
    next_date = datetime.strptime(rebal_date, '%Y-%m-%d')
    #next_date = str(next_date.month)+'/'+str(next_date.day)+'/'+str(next_date.year)+' 03:30:00 PM'
    next_detail['date'] = next_date
    next_detail.reset_index(drop=True, inplace=True)
    next_detail['group_by'] = ''
    next_detail = pd.merge(next_detail,
             price_d.loc[price_d.date==rebal_date, ['date', 'itemcode']],
             left_on=['date', 'itemcode'],
             right_on=['date', 'itemcode'], how='left')
    next_detail['username'] = username
    next_detail['userid'] = userid
    next_detail['original'] = 'Rebal'
    next_detail = next_detail.rename(columns={'weights':'wt'})
    next_detail = next_detail[['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
           'itemname', 'asset_class', 'date', 'userid', 'username', 'group_by',
           'original', 'wt']]

    return next_detail

In [36]:
detail

,date,itemcode,weights,itemname,price,asset_class,quantity,value,original
0,2020-02-03,A245340,0.2387,TIGER 미국다우존스30,17050,Equity,14,238700,Y
1,2020-02-03,A251350,0.232425,KODEX 선진국MSCI World,15495,Equity,15,232425,Y
2,2020-02-03,A148070,0.122404,KOSEF 국고채10년,122404,Fixed Income,1,122404,Y
3,2020-02-03,A278540,0.1647,KODEX MSCI Korea TR,9150,Equity,18,164700,Y
4,2020-02-03,A102780,0.04194,KODEX 삼성그룹,6990,Equity,6,41940,Y
5,2020-02-03,A273130,0,KODEX 종합채권(AA-이상)액티브,108930,Fixed Income,0,0,Y
6,2020-02-03,A237370,0.01039,KODEX 배당성장채권혼합,10390,Alternative,1,10390,Y
7,2020-02-03,C000001,0.189441,Cash,1,Cash,189441,189441,Y


In [118]:
next_detail = copy.deepcopy(detail)
all_the_nexts = pd.DataFrame(columns=next_detail.columns)
for dt in dates:
    price_d = price_db.loc[price_db.date==dt, ['date', 'price', 'itemcode']].reset_index(drop=True)
    if dt in rebal_dates[1:]:
        # 리밸런싱한다.
        new_port = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date==dt), ['date', 'itemcode', 'weights', 'itemname', 'price', 'asset_class']]
        next_detail = rebalance(rebal_date=dt, price_d=price_d, detail=copy.deepcopy(next_detail), new_port=new_port)
    else:
        # 리밸런싱 일자가 아니면, 새로운 종가만 업데이트하고 종목별 시가평가액(value=price*quantity)만 업데이트한다.
        next_date = datetime.strptime(dt, '%Y-%m-%d')
        #next_date = str(next_date.month)+'/'+str(next_date.day)+'/'+str(next_date.year)+' 09:00:00 AM'
        next_detail = copy.deepcopy(detail)
        next_detail['date'] = next_date
        next_detail = pd.merge(next_detail,
                             price_d.loc[price_d.date == next_date,
                                         ['date', 'itemcode', 'price']],
                             left_on=['date', 'itemcode'],
                             right_on=['date', 'itemcode'],
                             suffixes=('_cur', '_next'),
                             how='left')
        next_date = datetime.strptime(str(next_date.month)+'/'+str(next_date.day)+'/'+str(next_date.year), '%m/%d/%Y')
        next_detail['date'] = next_date
        next_detail.drop(['price_cur'], axis=1, inplace=True)
        next_detail.rename({'price_next':'price'}, axis=1, inplace=True)
        next_detail.price = next_detail.price.fillna(1)

        # 종목별 평가액과 비중 업데이트
        next_detail['value'] = next_detail['price']*next_detail['quantity']
        next_detail['wt'] = next_detail['value']/next_detail['value'].sum()
        next_detail['original'] = 'N'        
        #next_detail.loc[next_detail.itemcode=='C000001', 'price'] = 1
#        next_detail = next_detail[['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
#               'itemname', 'asset_class', 'date', 'userid', 'username', 'group_by',
#               'original', 'wt']]

    all_the_nexts = pd.concat((all_the_nexts, next_detail))

# 불필요한 컬럼 및 행 삭제
all_the_nexts = all_the_nexts.drop(['weights'], axis=1)
all_the_nexts = all_the_nexts.loc[all_the_nexts.quantity > 0]

Discrete allocation on 2020-02-23:{'A273130': 2, 'A214980': 2, 'A196230': 2, 'A114260': 1, 'A122260': 1, 'A069500': 2, 'A148070': 1, 'A251350': 1, 'A237370': 2, 'A278540': 1}
Funds remaining: 4510.00 KRW
Discrete allocation on 2020-03-14:{'A273130': 2, 'A114260': 4, 'A196230': 2, 'A148070': 1, 'A069500': 2, 'A278540': 2, 'A237370': 1}
Funds remaining: 6014.00 KRW
Discrete allocation on 2020-04-03:{'A114260': 3, 'A273130': 2, 'A122260': 1, 'A148070': 1, 'A069500': 4, 'A130730': 1, 'A237370': 2, 'A251350': 2}
Funds remaining: 4640.00 KRW
Discrete allocation on 2020-04-23:{'A114260': 4, 'A122260': 2, 'A273130': 2, 'A148070': 1, 'A069500': 3, 'A237370': 2, 'A251350': 1}
Funds remaining: 18757.00 KRW
Discrete allocation on 2020-05-13:{'A114260': 3, 'A122260': 2, 'A273130': 2, 'A148070': 1, 'A069500': 3, 'A130730': 1, 'A237370': 2}
Funds remaining: 1610.00 KRW
Discrete allocation on 2020-06-02:{'A114260': 3, 'A122260': 2, 'A273130': 2, 'A148070': 1, 'A069500': 3, 'A130730': 1, 'A237370': 2, 

In [133]:
all_the_nexts.date = all_the_nexts.date.map(lambda x:str(x.month)+'/'+str(x.day)+'/'+str(x.year)+ ' 4:00:00 PM')

In [135]:
all_the_nexts

,date,itemcode,itemname,price,asset_class,quantity,value,original,wt,cost_price,cost_value,userid,username,group_by
0,2/3/2020 4:00:00 PM,A245340,TIGER 미국다우존스30,17050,Equity,14,238700,N,0.2387,NaN,NaN,NaN,NaN,NaN
1,2/3/2020 4:00:00 PM,A251350,KODEX 선진국MSCI World,15495,Equity,15,232425,N,0.232425,NaN,NaN,NaN,NaN,NaN
2,2/3/2020 4:00:00 PM,A148070,KOSEF 국고채10년,122404,Fixed Income,1,122404,N,0.122404,NaN,NaN,NaN,NaN,NaN
3,2/3/2020 4:00:00 PM,A278540,KODEX MSCI Korea TR,9150,Equity,18,164700,N,0.1647,NaN,NaN,NaN,NaN,NaN
4,2/3/2020 4:00:00 PM,A102780,KODEX 삼성그룹,6990,Equity,6,41940,N,0.04194,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3/19/2021 4:00:00 PM,A148070,KOSEF 국고채10년,119780,Fixed Income,1,119780,N,0.103969,NaN,NaN,NaN,NaN,NaN
3,3/19/2021 4:00:00 PM,A278540,KODEX MSCI Korea TR,13460,Equity,18,242280,N,0.2103,NaN,NaN,NaN,NaN,NaN
4,3/19/2021 4:00:00 PM,A102780,KODEX 삼성그룹,9860,Equity,6,59160,N,0.051351,NaN,NaN,NaN,NaN,NaN
6,3/19/2021 4:00:00 PM,A237370,KODEX 배당성장채권혼합,11750,Alternative,1,11750,N,0.010199,NaN,NaN,NaN,NaN,NaN


In [121]:
all_the_nexts.loc[:,['date', 'wt', 'value', 'asset_class']].sort_values(
                                          ['date'], ascending=True)

,date,wt,value,asset_class
0,2020-02-03,0.2387,238700,Equity
1,2020-02-03,0.232425,232425,Equity
2,2020-02-03,0.122404,122404,Fixed Income
3,2020-02-03,0.1647,164700,Equity
4,2020-02-03,0.04194,41940,Equity
...,...,...,...,...
1,2021-03-19,0.231106,266250,Equity
2,2021-03-19,0.103969,119780,Fixed Income
3,2021-03-19,0.2103,242280,Equity
4,2021-03-19,0.051351,59160,Equity


In [124]:
all_the_generals = all_the_nexts.loc[:,['date', 'wt', 'value', 'asset_class']].sort_values(
                                          ['date'], ascending=True).groupby([
                                              'date', 'asset_class'
                                          ]).sum().reset_index(drop=False)

In [92]:
dt = datetime.datetime.strptime(current_date, '%Y-%m-%d')
date = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 9:00:00 AM'

all_the_nexts.date

0   2020-02-03
1   2020-02-03
2   2020-02-03
3   2020-02-03
4   2020-02-03
       ...    
2   2021-03-19
3   2021-03-19
4   2021-03-19
6   2021-03-19
7   2021-03-19
Name: date, Length: 2911, dtype: datetime64[ns]

In [88]:
all_the_generals['userid'] = userid

In [136]:
datetime.strptime('2021-2-3', '%Y-%m-%d').strftime('%Y-%m-%d')

'2021-02-03'

In [85]:
all_the_nexts.loc[all_the_nexts.date==dates[0]]

,date,itemcode,itemname,price,asset_class,quantity,value,original,wt,cost_price,cost_value,userid,username,group_by
0,2020-02-03,A245340,TIGER 미국다우존스30,17050,Equity,14,238700,N,0.2387,NaN,NaN,NaN,NaN,NaN
1,2020-02-03,A251350,KODEX 선진국MSCI World,15495,Equity,15,232425,N,0.232425,NaN,NaN,NaN,NaN,NaN
2,2020-02-03,A148070,KOSEF 국고채10년,122404,Fixed Income,1,122404,N,0.122404,NaN,NaN,NaN,NaN,NaN
3,2020-02-03,A278540,KODEX MSCI Korea TR,9150,Equity,18,164700,N,0.1647,NaN,NaN,NaN,NaN,NaN
4,2020-02-03,A102780,KODEX 삼성그룹,6990,Equity,6,41940,N,0.04194,NaN,NaN,NaN,NaN,NaN
6,2020-02-03,A237370,KODEX 배당성장채권혼합,10390,Alternative,1,10390,N,0.01039,NaN,NaN,NaN,NaN,NaN
7,2020-02-03,C000001,Cash,1,Cash,189441,189441,N,0.189441,NaN,NaN,NaN,NaN,NaN


In [90]:
datetime.strptime('2020-2-3', '%Y-%m-%d').strftime('%Y-%m-%d')

'2020-02-03'

## 자산군별 바 차트

In [50]:
import plotly.express as px

In [51]:
detail_day1 = all_the_nexts.loc[all_the_nexts.date==current_date]

In [68]:
v = detail_day1.loc[:, ['wt', 'itemname']].groupby(
                    'itemname').sum().sort_values('wt', ascending=False).reset_index()

In [59]:
a = detail_day1.loc[:, ['wt', 'asset_class']].groupby(
                    'asset_class').sum().sort_values('wt', ascending=False).reset_index()

In [60]:
asset_class = ['Fixed Income', 'Alternative', 'Cash', 'Equity']

In [61]:
df_ac = pd.DataFrame(asset_class, columns=['asset_class'])

In [64]:
df_by_asset_class = a.merge(df_ac, left_on='asset_class', right_on='asset_class', how='right').sort_values('wt', ascending=False).reset_index(drop=True)

In [65]:
df_by_asset_class

,asset_class,wt
0,Equity,0.677765
1,Cash,0.189441
2,Fixed Income,0.122404
3,Alternative,0.010390


In [69]:
fig1 = px.pie(v, names = 'itemname', values = 'wt', title = "추천 포트폴리오", color_discrete_sequence=px.colors.qualitative.Set3)
fig1.show()

In [70]:
df_by_asset_class.loc[:, ['asset_class', 'wt']]

,asset_class,wt
0,Equity,0.677765
1,Cash,0.189441
2,Fixed Income,0.122404
3,Alternative,0.010390


In [71]:
fig2 = px.bar(df_by_asset_class, y = 'wt', x = 'asset_class', title = '자산군별 비중',
       labels = {'asset_class': '자산군', 'wt': '비중'},
       orientation = 'v', color = "asset_class", color_continuous_scale = 'darkmint',
        template = 'plotly_dark')
fig2.show()